### 1.1

The transition probability matrix A is constructed to model the robot’s motion within the grid-based environment under strict physical constraints. Transitions are allowed only between neighboring cells in the four cardinal directions, while diagonal movements and transitions through obstacles are explicitly excluded. For each valid state, all feasible outgoing transitions toward accessible neighboring states are assigned equal probability, ensuring a uniform redistribution of probability mass among admissible moves. This construction guarantees that the resulting matrix is stochastic and accurately reflects the local and non-deterministic nature of the robot’s motion within the maze.

The transition matrix A represents the motion dynamics of the robot within the maze by encoding, for each state, the probability of transitioning to any other state in a single time step. Each row of the matrix corresponds to a starting state, while each column represents a possible destination state. Nonzero entries appear only for physically reachable neighboring cells, indicating that the robot can move exclusively toward accessible adjacent positions. The equality of the transition probabilities reflects the assumption of a uniform random choice among all feasible movements.

The transition matrix $A$ defines the probability of moving from one hidden state to another in the Hidden Markov Model. Each element $A_{ij}$ represents the probability of transitioning from state $i$ to state $j$ in a single time step:

$$
A_{ij} = P(X_{t+1} = j \mid X_t = i)
$$

where $X_t$ is the hidden state at time $t$. In our grid-based environment, transitions are allowed only to physically adjacent and accessible cells (North, South, East, West), while diagonal moves and moves through obstacles are forbidden.

If state $i$ has $n_i$ valid neighboring cells, each feasible transition has equal probability:

$$
A_{ij} =
\begin{cases} 
\dfrac{1}{n_i}, & \text{if state } j \text{ is a valid neighbor of } i \\
0, & \text{otherwise}
\end{cases}
$$




### 1.2

The emission probability matrix B encodes the relationship between the robot’s position and its sensory observations. For each state in the grid, the robot senses whether the neighboring cells to the North, South, East, and West are accessible. Each possible observation, previously defined in the variable `vocabulary`, corresponds to a specific combination of accessible directions. The matrix B is constructed such that for a given state, the column corresponding to the matching observation is assigned a probability of one, while all other entries remain zero. This deterministic assignment reflects the assumption that the robot’s sensors perfectly detect the local configuration of free and blocked neighboring cells.

The emission matrix $B$ defines the probability of observing a certain sensor reading given the hidden state of the robot. Each element $B_{ik}$ represents the probability of observing $O_k$ when the robot is in state $i$:

$$
B_{ik} = P(O_t = k \mid X_t = i)
$$

where $O_t$ is the observation at time $t$. The robot senses whether the neighboring cells to the North, South, East, and West are accessible. Each combination of accessible directions corresponds to a predefined observation in the `vocabulary`.

The matrix is deterministic:

$$
B_{ik} =
\begin{cases} 
1, & \text{if observation } k \text{ matches the accessible neighbors of state } i \\
0, & \text{otherwise}
\end{cases}
$$

This reflects perfect sensing: the robot can fully detect which neighboring cells are free.

### 1.3

We sample and visualize a sequence of hidden states $Z$ and observations $Y$ of length $T = 20$ from the previously defined Hidden Markov Model. The initial state $Z_0$ is drawn from the uniform distribution $\pi$ over all admissible states, reflecting equiprobable starting positions. Subsequent states $Z_t$ for $t > 0$ are sampled according to the transition matrix $A$, which encodes the probabilities of moving between accessible neighboring cells in the grid:

$$
Z_0 \sim \pi, \quad
Z_t \sim P(X_t \mid X_{t-1}) = A_{Z_{t-1}}
$$

For each hidden state $Z_t$, the corresponding observation $Y_t$ is deterministically determined using the emission matrix $B$, which encodes the robot's local sensory perception of free neighboring cells:

$$
Y_t = O_k \quad \text{such that } B_{Z_t, k} = 1
$$

This procedure generates sequences $(Z, Y)$ that are consistent with the model assumptions: the robot moves only to accessible neighboring states and observes exactly the configuration of free adjacent cells at each step.


### likelihood

We compute the likelihood of the observed sequence $Y$ using the Forward algorithm, which efficiently sums over all possible hidden state sequences. Let $\alpha_t(i)$ denote the forward probability of observing the first $t$ observations and ending in hidden state $i$ at time $t$:

$$
\alpha_0(i) = \pi_i \, B_{iY_0}
$$

$$
\alpha_t(j) = \sum_{i=1}^{N} \alpha_{t-1}(i) \, A_{ij} \, B_{jY_t}, \quad t = 1, \dots, T-1
$$

where $A_{ij}$ is the transition probability from state $i$ to $j$, $B_{jY_t}$ is the emission probability of observation $Y_t$ given state $j$, and $\pi_i$ is the initial state probability. 

The likelihood of the observation sequence $Y$ is then obtained by summing the forward probabilities at the final time step:

$$
P(Y) = \sum_{i=1}^{N} \alpha_{T-1}(i)
$$

This approach avoids enumerating all possible state sequences, providing an efficient way to compute $P(Y)$ for a Hidden Markov Model.


### decoding

We use the Viterbi algorithm to decode the most probable sequence of hidden states $Z_{\text{vit}}$ given the observed sequence $Y$. Let $\delta_t(i)$ denote the highest probability of a state sequence ending in hidden state $i$ at time $t$ that produces the first $t$ observations:

$$
\delta_0(i) = \pi_i \, B_{iY_0}
$$

$$
\delta_t(j) = \max_{i=1,\dots,N} \left[ \delta_{t-1}(i) \, A_{ij} \right] B_{jY_t}, \quad t = 1, \dots, T-1
$$

During the recursion, we store the backpointers:

$$
\psi_t(j) = \arg\max_{i=1,\dots,N} \left[ \delta_{t-1}(i) \, A_{ij} \right]
$$

After computing all $\delta_t(j)$ and $\psi_t(j)$, the most probable state sequence is obtained by backtracking from the state with the highest $\delta_{T-1}(i)$:

$$
Z_{\text{vit}}[T-1] = \arg\max_i \delta_{T-1}(i)
$$

$$
Z_{\text{vit}}[t] = \psi_{t+1}(Z_{\text{vit}}[t+1]), \quad t = T-2, \dots, 0
$$

This procedure provides the single most likely sequence of hidden states that explains the observed data according to the HMM.
